# TP 3 - Comment casser le LCG

Pour ce dernier TP on va implémenter la solution algrébrique vu en cours pour déterminer les paramètres d'un LCG à partir de quelques nombres générés.

On rappelle la relation de récurrence linéaire pour le LCG
$X_{n} = a*X_{n-1} + b \pmod{m}$

On va calculer $Y_{n}$ et $Z_{n}$
$Y_{n} = X_{n} - X_{n-1}$
Et
$Z_{n} = Y_{n}Y_{n+2} - Y_{n+1}^{2}$
Avec un nombre de $Z_{n}$ suffisant on peut déterminer $m$ qui est le GCD, plus grand dénominateur commun de tous les $Z_{n}$

A partir de la valeur du modulo $m$ on peut calculer $a$ et $b$ à partir des deux équations suivantes

$\begin{cases}
    X_{1} = a*X_{0} + b \pmod{m} \\
    X_{2} = a*X_{1} + b \pmod{m} \\
\end{cases}$
,
$a = \frac{X_{2} - X_{1}}{X_{1} - X_{0}} \pmod{m}$
,
$b = X_{1} - a*X_{0} \pmod{m}$



In [38]:
# récupère notre LCG avec une sortie non modifiée
class LinearCongruentialGenerator:
    def __init__(self, is_bit=False, seed=1, a=1664525, c=1013904223, m=2**32):
        self.is_bit = is_bit
        self.state = int(seed)
        self.a = a
        self.c = c
        self.m = m

    def next(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state

## Implémentation du modulo inverse
On a besoin d'une implémentation du modulo inverse pour le calcule de $a$
Pour se faire, on doit d'abord implémenter l'extension du gcd  $ax+by=gcd(a,b)$ qui détermine les valeurs x et y. On pourra utiliser ce [lien](https://brilliant.org/wiki/extended-euclidean-algorithm/) pour comprendre le mécanisme.

In [16]:
def egcd(a, b):
    # votre code
    if a == 0:
        return b, 0, 1
    else:
        g, y, x = egcd(b % a, a)
        return g, x - (b // a) * y, y

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [17]:
# Exécuter cette cellule afin de vérifier que l'implémentation est correct
g_test, x_test, y_test = egcd(102, 38)
assert g_test == 2
assert x_test == 3
assert y_test == -8
print("Test successful !!!")

Test successful !!!


## Implémentation du reste de la solution

In [45]:
from math import gcd


def solve(Xn):

    # votre code
    # Calcul Yn
    Yn = [X1 - X0 for X0, X1 in zip(Xn, Xn[1:])]
    # Calcul Zn
    Zn = [Y2*Y0 - Y1*Y1 for Y0, Y1, Y2 in zip(Yn, Yn[1:], Yn[2:])]

    # Trouver le gcd de Zn pour avoir le modulo m
    m = Zn[0]
    for x in Zn[1:]:
        m = gcd(m, x)

    # Calculer a et b, astuce : ajouter m a Yn afin de s'assurer d'avoir une valeur négative avant d'effectuer le modulo inverse sans changer le résultat
    # Ce n'est pas necessaire pour le modulo % car les cas négatifs sont déjà pris en compte par python
    a = Yn[1] * modinv(Yn[0] + m, m) % m
    b = (Xn[1] - a * Xn[0]) % m

    print(f"Les valeurs des paramètres trouvés sont: \n a = {a}, b = {b} et m = {m}")
    return a, b, m

In [50]:
# Exécuter cette cellule afin de vérifier que l'implémentation est correct
LCG = LinearCongruentialGenerator(seed=42)
Xn = []
for i in range(5):
    Xn.append(LCG.next())

a, b, m = solve(Xn)
assert a == 1664525
assert b == 1013904223
assert m == 2**32
print("Test successful !!!")

Les valeurs des paramètres trouvés sont: 
 a = 1664525, b = 1013904223 et m = 4294967296
Test successful !!!


**On a réussi à retrouver les paramètres de notre LCG !!!
 Avec seulement 5 valeurs qui est le nombre minimal pour avoir au moins 2 valeurs de** $Z_{n}$